In [15]:
import numpy as np
from PIL import Image
import json
import os
import time
from glob import glob


Classes = ["D00","D10","D20","D40"]
Names = ["Czech","India","Japan"]
Box = [600,720,600]

In [16]:
def nms(dets, thresh):
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep

In [17]:
with open('ensemble1/base_model.json', 'r') as f:
    D = json.load(f)


Ensemble_D = {}


mods = glob('ensemble1/model_*')

for mod_idx in range(len(mods)):
    with open(mods[mod_idx], 'r') as f:
        Ensemble_D[mod_idx] = json.load(f)

## Test 1

In [18]:
!rm "ensemble1.txt"
con = [0.22,0.16,0.11]
con3 = [0.7,0.7,0.6]
con4 = [0.95,0.95,0.4]
nm = [1,1,1]


submission = list()
dets = list()

for count in range(len(D)):
    line = list()
    boxes = list()

    name = D[count]['filename'].split("/")[-1]
    line.append(name)
    idd = Names.index(name.split("_")[0])

    if (len(D[count]['objects']))!=0:
        for bound in (D[count]['objects']):
            if bound['confidence'] >=con[idd]:
                idx = Classes.index(bound['name']) + 1
                y = bound['relative_coordinates']['center_y']*Box[idd]
                x = bound['relative_coordinates']['center_x']*Box[idd]
                h = bound['relative_coordinates']['height']*Box[idd]*0.5
                w = bound['relative_coordinates']['width']*Box[idd]*0.5

                boxes.append([int(x-w),int(y-h),int(x+w),int(y+h),bound['confidence'],idx])
    
    for mod_idx in range(len(mods)):
        if (len(Ensemble_D[mod_idx][count]['objects']))!=0:
            for bound in (Ensemble_D[mod_idx][count]['objects']):
                if bound['confidence'] >=con3[idd]:
                    idx = Classes.index(bound['name']) + 1
                    y = bound['relative_coordinates']['center_y']*Box[idd]
                    x = bound['relative_coordinates']['center_x']*Box[idd]
                    h = bound['relative_coordinates']['height']*Box[idd]*0.5
                    w = bound['relative_coordinates']['width']*Box[idd]*0.5

                    boxes.append([int(x-w),int(y-h),int(x+w),int(y+h),bound['confidence'],idx])
    
    
    if len(boxes) > 0:
        boxes = np.array(boxes).reshape(-1,6)            
        ore = nms(boxes,nm[idd])
        
        for o in ore:
            line.extend([boxes[o,5],boxes[o,0],boxes[o,1],boxes[o,2],boxes[o,3]])
            

    text_file = open("ensemble1.txt", "a")
    for a in line[0:1]:
        text_file.write(str(a)+",")
    dets.extend(line[1:])


    for a in line[1:]:
        text_file.write('%d' %a)
        text_file.write(' ')

    text_file.write("\n")
    text_file.close()

## Test 2

In [19]:
with open('ensemble2/base_model.json', 'r') as f:
    D = json.load(f)


Ensemble_D = {}


mods = glob('ensemble2/model_*')

for mod_idx in range(len(mods)):
    with open(mods[mod_idx], 'r') as f:
        Ensemble_D[mod_idx] = json.load(f)

In [20]:
!rm "ensemble2.txt"
con = [0.22,0.16,0.11]
con3 = [0.7,0.7,0.6]

nm = [1,1,1]


submission = list()
dets = list()

for count in range(len(D)):
    line = list()
    boxes = list()

    name = D[count]['filename'].split("/")[-1]
    line.append(name)
    idd = Names.index(name.split("_")[0])

    if (len(D[count]['objects']))!=0:
        for bound in (D[count]['objects']):
            if bound['confidence'] >=con[idd]:
                idx = Classes.index(bound['name']) + 1
                y = bound['relative_coordinates']['center_y']*Box[idd]
                x = bound['relative_coordinates']['center_x']*Box[idd]
                h = bound['relative_coordinates']['height']*Box[idd]*0.5
                w = bound['relative_coordinates']['width']*Box[idd]*0.5

                boxes.append([int(x-w),int(y-h),int(x+w),int(y+h),bound['confidence'],idx])
    
    for mod_idx in range(len(mods)):
        if (len(Ensemble_D[mod_idx][count]['objects']))!=0:
            for bound in (Ensemble_D[mod_idx][count]['objects']):
                if bound['confidence'] >=con3[idd]:
                    idx = Classes.index(bound['name']) + 1
                    y = bound['relative_coordinates']['center_y']*Box[idd]
                    x = bound['relative_coordinates']['center_x']*Box[idd]
                    h = bound['relative_coordinates']['height']*Box[idd]*0.5
                    w = bound['relative_coordinates']['width']*Box[idd]*0.5

                    boxes.append([int(x-w),int(y-h),int(x+w),int(y+h),bound['confidence'],idx])
    
    
    if len(boxes) > 0:
        boxes = np.array(boxes).reshape(-1,6)            
        ore = nms(boxes,nm[idd])
        
        for o in ore:
            line.extend([boxes[o,5],boxes[o,0],boxes[o,1],boxes[o,2],boxes[o,3]])
            

    text_file = open("ensemble2.txt", "a")
    for a in line[0:1]:
        text_file.write(str(a)+",")
    dets.extend(line[1:])


    for a in line[1:]:
        text_file.write('%d' %a)
        text_file.write(' ')

    text_file.write("\n")
    text_file.close()